In [14]:
import requests
import pandas as pd
import time
from datetime import datetime
import json

class NaverShoppingCrawler:
    def __init__(self, client_id, client_secret):
        """네이버 쇼핑 API 크롤러 초기화"""
        self.client_id = client_id
        self.client_secret = client_secret
        self.base_url = "https://openapi.naver.com/v1/search/shop.json"
    
    def search_products(self, keyword, display=100, start=1):
        """상품 검색"""
        headers = {
            "X-Naver-Client-Id": self.client_id,
            "X-Naver-Client-Secret": self.client_secret
        }
        params = {
            "query": keyword,
            "display": display,
            "start": start,
            "sort": "sim"
        }
        try:
            res = requests.get(self.base_url, headers=headers, params=params, timeout=10)
            res.raise_for_status()
            data = res.json()
            if 'items' not in data:
                print(f"⚠️ 응답에 'items'가 없습니다: {data}")
                return None
            return data
        except Exception as e:
            print(f"❌ 요청 오류 ({keyword}): {e}")
            return None
    
    def extract_product_info(self, item):
        """상품 정보 정리"""
        def clean(t):
            return str(t).replace("<b>", "").replace("</b>", "").strip() if t else ""

        return {
            "title": clean(item.get("title")),
            "link": item.get("link", ""),
            "image": item.get("image", ""),
            "lprice": int(item.get("lprice", 0)) if item.get("lprice") else 0,
            "hprice": int(item.get("hprice", 0)) if item.get("hprice") else 0,
            "mall_name": item.get("mallName", "Unknown"),
            "product_id": item.get("productId", ""),
            "brand": item.get("brand", "Unknown"),
            "maker": item.get("maker", "Unknown"),
            "category1": item.get("category1", ""),
            "category2": item.get("category2", ""),
            "category3": item.get("category3", ""),
            "category4": item.get("category4", "")
        }

    def test_connection(self):
        """API 연결 테스트"""
        print("\n🔍 네이버 API 연결 테스트 중...")
        test = self.search_products("텀블러", display=1, start=1)
        if test and 'items' in test and len(test['items']) > 0:
            print("✅ API 연결 성공!")
            print(f"총 검색 결과: {test.get('total', 0)}개")
            print("\n샘플 데이터 예시:")
            print(json.dumps(test['items'][0], indent=2, ensure_ascii=False))
            return True
        else:
            print("❌ API 연결 실패! (Client ID/Secret, 권한, 쇼핑 API 설정 확인)")
            return False

    def crawl_keyword(self, keyword, max_items=500):
        """한 키워드 크롤링"""
        print(f"\n🔎 '{keyword}' 검색 시작...")
        all_items = []
        display = 100

        for start in range(1, max_items + 1, display):
            data = self.search_products(keyword, display, start)
            if not data or not data.get("items"):
                print("⚠️ 더 이상 데이터 없음.")
                break
            for item in data["items"]:
                all_items.append(self.extract_product_info(item))
            print(f"✅ {len(data['items'])}개 수집 완료 (총 {len(all_items)}개)")
            time.sleep(0.2)
            if len(data["items"]) < display:
                break

        return pd.DataFrame(all_items)


# === 실행 ===
if __name__ == "__main__":
    print("="*70)
    print("🛒 네이버 쇼핑 상품 데이터 크롤러")
    print("="*70)

    CLIENT_ID = input("\n👉 네이버 Client ID 입력: ").strip()
    CLIENT_SECRET = input("👉 네이버 Client Secret 입력: ").strip()

    if not CLIENT_ID or not CLIENT_SECRET:
        print("❌ Client ID 또는 Secret이 비어 있습니다.")
        exit()

    crawler = NaverShoppingCrawler(CLIENT_ID, CLIENT_SECRET)

    # API 연결 테스트
    if not crawler.test_connection():
        exit()

    keywords = ["텀블러", "보온병", "스텐텀블러"]
    all_data = []

    for kw in keywords:
        df = crawler.crawl_keyword(kw, max_items=300)
        if not df.empty:
            df["search_keyword"] = kw
            all_data.append(df)

    if not all_data:
        print("\n❌ 수집된 데이터가 없습니다.")
        exit()

    final = pd.concat(all_data, ignore_index=True)
    final.drop_duplicates(subset=["product_id"], inplace=True)
    final = final[final["lprice"] > 0]

    file_name = f"naver_shopping_{datetime.now().strftime('%Y%m%d_%H%M')}.csv"
    final.to_csv(file_name, index=False, encoding="utf-8-sig")

    print("\n🎉 완료!")
    print(f"📁 저장된 파일: {file_name}")
    print(f"총 상품 수: {len(final):,}개")
    print(f"평균 가격: {final['lprice'].mean():,.0f}원")
    print("\n상위 쇼핑몰:")
    print(final['mall_name'].value_counts().head(5))


🛒 네이버 쇼핑 상품 데이터 크롤러



🔍 네이버 API 연결 테스트 중...
✅ API 연결 성공!
총 검색 결과: 3188285개

샘플 데이터 예시:
{
  "title": "[스타벅스] SS 탱크 <b>텀블러</b> 503ml",
  "link": "https://smartstore.naver.com/main/products/11519149266",
  "image": "https://shopping-phinf.pstatic.net/main_8906365/89063659659.14.jpg",
  "lprice": "31500",
  "hprice": "",
  "mallName": "스타벅스",
  "productId": "89063659659",
  "productType": "2",
  "brand": "스타벅스",
  "maker": "",
  "category1": "생활/건강",
  "category2": "주방용품",
  "category3": "잔/컵",
  "category4": "텀블러"
}

🔎 '텀블러' 검색 시작...
✅ 100개 수집 완료 (총 100개)
✅ 100개 수집 완료 (총 200개)
✅ 100개 수집 완료 (총 300개)

🔎 '보온병' 검색 시작...
✅ 100개 수집 완료 (총 100개)
✅ 100개 수집 완료 (총 200개)
✅ 100개 수집 완료 (총 300개)

🔎 '스텐텀블러' 검색 시작...
✅ 100개 수집 완료 (총 100개)
✅ 100개 수집 완료 (총 200개)
✅ 100개 수집 완료 (총 300개)

🎉 완료!
📁 저장된 파일: naver_shopping_20251103_2105.csv
총 상품 수: 705개
평균 가격: 32,702원

상위 쇼핑몰:
mall_name
네이버         223
스탠리 코리아      40
쿠팡           22
이룸 EROUM     18
스타벅스         15
Name: count, dtype: int64


In [15]:
import pandas as pd
import glob

#naver_shopping으로 시작하는 모든 csv찾기
files = glob.glob("/Users/sebokoh/데이터분석연습/프로젝트(데이터분석3가지)/naver_shopping_20251030_1418.csv")
print(f"발견된 파일: {files}")

if files:
    #가장 최근 파일 읽기
    latest = sorted(files)[-1]
    df = pd.read_csv(latest)
    print(f"\n파일: {latest}")
    print(f"행 수: {len(df):,}")
    print(f"열 수: {len(df.columns)}")
    print(f"\n컬럼: {list(df.columns)}")
    print(f"\n샘플 데이터:")
    print(df.head(3))

발견된 파일: ['/Users/sebokoh/데이터분석연습/프로젝트(데이터분석3가지)/naver_shopping_20251030_1418.csv']

파일: /Users/sebokoh/데이터분석연습/프로젝트(데이터분석3가지)/naver_shopping_20251030_1418.csv
행 수: 700
열 수: 14

컬럼: ['title', 'link', 'image', 'lprice', 'hprice', 'mall_name', 'product_id', 'brand', 'maker', 'category1', 'category2', 'category3', 'category4', 'search_keyword']

샘플 데이터:
                                        title  \
0        대용량 텀블러 퀜처 보온 보냉텀블러 빨대 손잡이텀블러 대형 차량용   
1       대용량 텀블러 빨대텀블러 보온보냉텀블러 스텐 손잡이 운동 기념품 소   
2  스탠리 스탠리1913 아이스플로우 플립스트로 2.0 텀블러 591ml, 1개   

                                                link  \
0  https://smartstore.naver.com/main/products/846...   
1  https://smartstore.naver.com/main/products/870...   
2  https://search.shopping.naver.com/catalog/5525...   

                                               image  lprice  hprice  \
0  https://shopping-phinf.pstatic.net/main_860141...   15900       0   
1  https://shopping-phinf.pstati

In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns   
from collections import Counter
import re
import warnings
import sys
warnings.filterwarnings('ignore')

# 한글 폰트 설정
try:
    plt.rcParams['font.family'] = 'AppleGothic' # 맥
except:
    print(" 한글 폰트 설정 실패. 그래프에서 한글이 깨질 수 있습니다.") 
    
plt.rcParams['axes.unicode_minus'] = False   

class TumblerAnalyzer:
    def __int__(self, csv_file):
        """CSV 파일 로드"""
        print("="*60)
        print("데이터 로딩 중...")
        print("="*60)
        
        try:
            self.df = pd.read.csv(csv_file, encoding=utf-8-sig)
            print(f" 파일 로드 성공: {csv_file}")
            print(f" 초기 데이터: {len(self.df):,}행 x {len(self.df.columns)}열") 
            
            #컬럼 확인
            print(f"\n컬럼 목록: {len(self.df.columns)}") 
            
            self.clean_data() 
            
        except FileNotFoundError:
            print(f"파일을 찾을 수 없습니다: {csv_file}") 
            print("현재 디렉토리의 CSV의 파일을 확인해 주세요.") 
            sys.exit(1)
        except Exception as e:
            print(f"에러 발생: {e}")
            sys.exit(1)
            
    def clean_data(self):
        """데이터 전처리"""
        print("\n" + "="*60)
        print(" 데이터 전처리 시작")
        print("="*60) 
        
        #중복 제거
        before = len(self.df) 
        if 'product_id' in self.df.columns:
            self.df = self.df.drop_duplicates(subset='product_id')
            print(f"중복 제거: {before:,} -> {len(self.df):,} (-{before - len(self.df):,}개)") 
            
        # lprice가 문자열인 경우
        if self.df['lprice'].dtype == 'object':
            self.df['lprice'] = pd.to_numeric(self.df['lprice'], errors='coerce') 
            print(f"가격 데이터 타입 변환 완료")
            
        # 가격이 0이거나 NaN인  데이터 제거
        before = len(self.df)  
        self.df = self.df[self.df['lprice'].notna() & (self.df['lprice'] > 0)]
        print(f" 가격 데이터 타입 변환 완료")
        
        #가격 이상치 제거 (5,000원 미만, 200,000원 초과) 
        before = len(self.df) 
        self_df = self.df[(self.df['lprice'] >= 5000) & (self.df['lprice'] <= 200000)]  
        print(f" 가격 이상치 제거 (5천~20만원): ") 
        
        # 용량 추출
        self.df['capacity'] = self.df['title'].apply(self.extract_capacity)
        capacity_count = self.df['capacity'].notna().sum()
        print(f" 용량 추출 완료: {capacity_count:,}개 ({capacity_count/len(self.df)*100:.1}%)") 
        
        #브랜드 분류
        self.df['brand_type'] = self.df['brand'].apply(self.catagorize_brand)
        print(f"브랜드 분류 완료!")
        
        # 가격대 구간
        self.df['price_range'] = pd.cut(self.df['lprice'],
                                        bins=[0, 10000, 20000, 30000, 50000, 200000],
                                        labels=['1만원 이하', '1-2만원', '2-3만원', '3-5만원', '5만원 이상'])
        
        print(f"\n 전처리 완료: {len(self.df):,)}개 상품")
        print("="*60)   

In [17]:
def extract_capacity(self, title):
    """상품명에서 용량 추출"""
    if pd.isna(title):
        return None
    
    title = str(title)
    
    # 350ml, 500ml, 1L 등
    ml_pattern = re.search(r'(\d+)ml', title, re.IGNORECASE)
    l_pattern = re.search(r'(\d+\.?\d*)L', title, re.IGNORECASE)
    
    if ml_pattern:
        ml = int(ml_pattern.group(1))
        return ml
    elif l_pattern:
        return int(float(l_pattern.group(1))*1000)
    return None

def categorize_brand(self, brand):
    """브랜드 분류"""
    known_brand = ['스탠리', 'stanley', '써모스', 'thermos', '타이거', 
                       '락앤락', '코멧', '빙그레', 'contigo', '콘티고', 
                       'yeti', '예티', 'hydroflask', '하이드로']
    
    if pd.isna(brand) or brand == 'Unknown' or brand == '':
        return '기타/노브랜드'
    
    brand_lower = str(brand).lower()
    for kb in known_brand:
        if kb.lower() in brand_lower():
            return '유명브랜드'
        
    return '기타/노브랜드'
    
def basic_statistics(self):
    """기본 통계 분석""" 
    print("\n" + "="*60) 
    print(" 기본 통계")
    print("="*60)
    
    print(f"\n총 상품 수: {len(self.df):,}개")
    print(f"\n 가격 통계:")
    print(f" 평균 가격: {self.df['lprice'].mean():,.0f}원")  
    print(f" 중앙 값: {self.df['lprice'].median():,.0f}원")    
    print(f" 최저가: {self.df['lprice'].min():,.0f}원")    
    print(f" 최고가: {self.df['lprice'].max():,.0f}원")    
    print(f" 표준편차: {self.df['lprice'].std():,.0f}원")
    
    return self.df.describe()   

def price_range_analysis(self):
    """가격대별 분석"""
    print("\n" + "="*60)
    print(" 가격대별 분포")
    print("="*60) 
    
    price_dist = self.df['price_range'].value_counts().sort_index()
    
    print()
    for price, count in price_dist.item():
        percentage = (count/len(self.df)*100)
        bar_length = int(percentage/ 2)
        bar = '█' * bar_length
        print(f"{price:15s}: {count:5,}개 ({percentage:5.1f}%) {bar}") 
        
    # 시각화
    try:
        plt.figure(figsize=(12, 6))
        ax = price_dist.plot(kind='bar', color='#3498db', edgecolor='black', alpha=0.8) 
        plt.title('가격대별 제품 분포', fontsize=18, fontwight='bold', pad=20) 
        plt.xlabel('가격대', fondsize=14, fontweigh='bold')
        plt.ylabel('제품 수', fondsize=14, fontweigh='bold')
        plt.xticks(rotation=45, ha='right')
        
        # 막대 위의 값 표시
        for i, v in enumerate(price_dist.values):
            ax.text(i, v + max(price_dist.values)*0.01, f'{v:,}개',
                ha='center', fontweigh='bold')
                
        plt.tight_layout()
        plt.savefig('price_distribution.png', dpi=300, bbox_inches='tight')
        print("\n그래프저장: price_distribution.png")
        plt.close()
    except Exception as e:
        print(f"\n그래프 저장 실패: {e}")
        
    return price_dist                  

In [18]:
def capacity_analsis(self):
    """용량별 분석"""
    print("\n" + "="*60)
    print(" 용량별 분석")
    print("="*60)
    
    # 용량 있는 데이터만
    df_with_capacity = self.df[self.df['capacity'].notna()].copy()
    
    if len(df_with_capacity) == 0:
        print(" 용량 정보가 있는 데이터가 없습니다.")
        return None
    
    print(f"용량 정보 있는 제품: {len(df_with_capacity):,}개, ({len(df_with_capacity)/len(self.df)*100:.1f}%")
    
    # 주요 용량대로 그룹화
    def categorize_capacity(cap):
        if cap < 400:
            return '소형 (<400ml)'
        elif cap < 600:
            return '중형 (400-600ml)'
        elif cap < 1000:
            return '대형 (600-1L)'
        else:
            return '특대형 (1L+)'
        
    df_with_capacity['capacity_group'] = df_with_capacity['capacity'].apply(categorize_capacity)
    
    # 용량별 통계
    print("\n 용량별 통계:")
    capacity_stats = df_with_capacity.groupby('capacity_group').agg({    
        'lprice': ['count', 'mean', 'median']
    }).round(0)
    capacity_stats.columns = ['제품수', '평균가격', '중간가격']
    print(capacity_stats.to_string())
    
    # 용량별 평균 가격 시각화     
    try:
        cap_price = df_with_capacity.groupby('capacity_group')['lprice'].mean().sort_values()
        
        plt.figure(figsize=(12, 6))
        ax = cap_price.plot(kind='barh', color='#e74c3c', edgecolor='black', alpha=0.8)
        plt.title('용량별 평균가격', fontsize=18, fontweigh='bold', pad=20)
        plt.xlabel('평균가격(원)', fontsize=14, fontweigh='bold')
        plt.title('용량', fontsize=14, fontweigh='bold')
        
        # 막대 끝에 값 표시
        for i, v in enumerate(cap_price.values):
            ax.text(v + max(cap_price.values)*0.01, i, f'{v:,.0f}원',
                    va='center',  fontweigh='bold')
            
        plt.tight_layout()
        plt.savefig('capacity_price.png', dpi=300, bbox_inches='tight') 
        print("\n그래프 저장: capacity_price.png")
        plt.close()
    except Exception as e:
        print(f"\n그래프 저장 실패 {e}")
        
    return df_with_capacity           
                 
                   

        
        

In [19]:
def brand_analysis(self):
        """브랜드 분석"""
        print("\n" + "="*60)
        print("🏷️ 브랜드 분석")
        print("="*60)
        
        brand_stats = self.df.groupby('brand_type').agg({
            'lprice': ['count', 'mean', 'median']
        }).round(0)
        brand_stats.columns = ['제품수', '평균가격', '중간가격']
        
        print("\n📊 브랜드 유형별 통계:")
        print(brand_stats.to_string())
        
        # 브랜드 프리미엄 계산
        known_data = self.df[self.df['brand_type'] == '유명브랜드']['lprice']
        unknown_data = self.df[self.df['brand_type'] == '기타/노브랜드']['lprice']
        
        if len(known_data) > 0 and len(unknown_data) > 0:
            known_avg = known_data.mean()
            unknown_avg = unknown_data.mean()
            premium = ((known_avg - unknown_avg) / unknown_avg) * 100
            
            print(f"\n💎 브랜드 프리미엄 분석:")
            print(f"   유명브랜드 평균: {known_avg:,.0f}원 ({len(known_data):,}개)")
            print(f"   기타브랜드 평균: {unknown_avg:,.0f}원 ({len(unknown_data):,}개)")
            print(f"   브랜드 프리미엄: {premium:.1f}% (유명브랜드가 {premium:.1f}% 더 비쌈)")
        
        # 상위 브랜드 분석
        print(f"\n🔝 상위 브랜드 TOP 10:")
        top_brands = self.df['brand'].value_counts().head(10)
        for i, (brand, count) in enumerate(top_brands.items(), 1):
            print(f"   {i:2d}. {brand:20s}: {count:4,}개")
        
        return brand_stats

In [20]:
def nall_analysis(self):
    """쇼핑몰 분석"""
    print("\n"+"="*60)
    print("주요 판매처 분석")
    print("="*60)
    
    top_malls = self.df['mall_name'].value_counts().head(15)
    
    print(f"\n상위 15개 쇼핑몰:")
    for i, (mall, count) in enumerate(top_mall.items(), 1):
        percentage = (count / len(self.df)) * 100
        bar_length = int(percentage/2)
        bar = '█' * bar_length
        print(f"{i:2d}. {mall:25s}: {count:4,}개 ({percentage:5.1f}%) {bar}")
        
    return top_malls

def keyword_analysis(self):
    """인기 키워드 분석"""
    print("\n" + "="*60)
    print("인기 키워드 분석")
    print("="*60)
    
    #제목에서 키워드 추출
    all_words = []
    for title in self.df['title']:
        if pd.notna(title):
            words = str(title).split()
            all_words.extend(words)
            
    # 불용어 제거
    stop_words = ['텀블러', '보온병', '물병', 'ml', 'L', '/', '-', '|', 
                     '(', ')', '[', ']', '<b>', '</b>', 'b']   
    filtered_words = [w for w in all_words
                      if w not in stop_words and len(w) and not w.isdigit()]  
    
    #빈도수 계산
    word_freq = Counter(filtered_words).most_common(30)
    
    print(f"\n TOP 30 인기 키워드:")
    for i, (word, freq) in enumerate(word_freq, 1):
        print(f"{i:2d}. {word:15s}: {freq:5,}회")
    
    return word_freq    
    
    
            
    

In [21]:
def find_sweet_spot(self):
    """스윗스팟 찾기(진입 추천하는 가격대)"""
    print("\n" + "="*60)
    print("시장 진입 전략 (스윗스팟)")
    print("="*60)
    
    #가격대별 경쟁 밀도
    price_bins = [0, 10000, 15000, 20000, 25000, 30000, 50000]
    price_labels = ['~1만', '1-1.5만', '1.5-2만', '2-2.5만', '2.5-3만', '3-5만', '5만+']
    
    self.df['detailed_price'] = pd.cut(self.df['lprice'], bins=price_bins, labels=price_labels)
    
    competition = self.df['detailed_price'].value_counts().sort_index()
    
    print("\n 가격대별 제품 수(경쟁 밀도):")
    max_count = competition.max()
    for price, count in competition.items():
        percentage = (count / len(self.df))*100
        intensity = '🔥'* int(count / max_count * 5) 
        print(f"{price:10s}: {count:5,}개 ({percentage:5.1f}%) {intensity}")
        
    #경쟁 적은 구간 찾기
    min_competition = competition.min()
    sweet_spots = competition[competition <= min_competition *1.5]
    
    print(f"\n 추천 진입 가격대(경쟁 상대적으로 적음):")
    for spot in sweet_spots.index:
        count = competition[spot]
        print(f"  {spot}원대 ({count:,}개 제품)")
    
    return competition    

In [22]:
def generate_report(self):
    """종합 리포트 생성"""
    print("\n" + "="*70)
    print(""*25 + "종합 분석 리포트")
    print("="*70)
    
    #모든 분석 실행
    self.basic_statistcs()
    self.price_range_analysis()
    self.capacity_analysis()
    self.brand_analysis()
    self.mall_analysis()
    self.keyword_analysis()
    competition = self.find_sweet_spot()
    
    #최종 추천
    print("\n" + "="*70)
    print(""*20 + " 창업자를 위한 최종추천")
    print("="*70)
    
    avg_price = self.df['lprice'].mean()
    median_price = self.df['lprice'].median()
    
    print(f"\n1️⃣ 추천 가격 설정:")
    print(f"  시장 평균: {avg_price:,.0f}원")
    print(f"  중앙값: {median_price:,.0f}원")
    print(f"  💡추천 진입 가격: {int(median_price * 0.9):,}원")
    print(f"    (중앙값의 90% = 가성비 어필 + 경쟁력 확보)")
    
    print(f"\n 2️⃣ 추천 제품 스펙")
    popular_capacity = self.df[self.df['capacity'].notna()]['capacity'].mode()
    if len(popular_capacity) > 0:
        print(f".  용량: {int(popular_capacity.values[0])}ml (가장 인기있는 용량)")
    else:
        print(f".   용량: 500ml  추천(표준 용량)")
    print(f"    브랜드: 기타/노브랜드로 시작 (진입장벽 낮은 시장)")
    print(f"    차별화: 색상 다양화, 각인 서비스, 독특한 디자인")
    
    print(f"\n 3️⃣ 예상 매출 시뮬레이션:")
    recommended_price = int(median_price * 0.9)
    daily_sales = 5
    monthy_revenue = recommended_price * daily_sales * 30
    net_profit = int(monthly_revenue * 0.3)  
    
    #--- 손익 분기점 계산 수정 시작(제미나이 도움으로 수정)--
    initial_investment = 100000  #초기 투자 10만원 가정
    profit_rate = 0.3            # 이익률 30% 가정
    
    #개당 순이익 = 판매가 * 이익률
    unit_net_profit = recommended_price * profit_rate
    
    #손익분기점 개수 = 초기 투자 / 개당 순이익
    # 개당 순이익이 0이 아닐 때만 계산합니다. 
    if unit_net_profit > 0:
        break_even_point = initial_investment / unit_net_profit
        # 결과 출력 시 소수점 이하를 버리고 정수로 표시합니다. 
        bep_output = f"약 {int(break_even_point):,}개 판매"
    else:
        bep_output = "순이익이 없어 계산 불가"
            
    print(f"  판매가: {recommended_price:,}원")
    print(f"  하루 판매: {daily_sales:,}개 (보수적 추정)")
    print(f"  월 매출: {monthy_revenue:,}원")
    print(f"  예상 순이익: {net_profit:,}원 (이익률 30%가정)")
    print(f"  손익 분기점: {bep_output} (초기 투자 {initial_investment}원 가정)") 
    # 손익분기점 계산 수정 끝 (제미나이 도움 )    
        

In [23]:
# 실행
if __name__ == "__ㅡmain__":
    import sys
    
    #CSV 파일 경로
    if len(sys.argv) > 1:
        csv_file = sys.argv[1]
    else:
        #최근 생성된 naver_shopping_*.csv 파일 자동 찾기
        import glob
        csv_files = glob.glob("naver_shopping_*.csv")
        
        if csv_files:
            #가장 최근 파일 사용
            csv_file = sorted(csv_files, reverse=True)[0]
            print(f" 자동으로 파일 선택: {csv_file}\n")
        else:
            print(" CSV 파일을 찾을 수 없습니다.") 
            print("사용법: python tumbler_analysis.py <csv_file>")
            print("또는 naver_shopping_*.csv파일을 같은 폴더에 넣어 주세요")
            sys.exit(1)
            
    #분석 시작
    analyzer = TumblerAnalyzer(csv_file)  
    
    
    #종합 리포트 생성
    analyzer.generate_report()         